In [19]:
import datetime#import datetime module convert data to datetime64(ns)
import numpy as np
import pandas as pd#import pandas module(Better than csv module!)
import os
import glob
import sklearn as sk
from sklearn.model_selection import train_test_split
from statistics import mean

def resha(x):
    nptrain=np.array(x)
    print(nptrain.shape)
    nptrain=np.reshape(nptrain,(nptrain.shape[0]*nptrain.shape[1], nptrain.shape[2]))
    return nptrain
    
def savenp(x,y):
    train_x,x_test,train_y,y_test = train_test_split(x,y,test_size=0.25,random_state=42)
    stock_name=path[12:21]
    scaler = sk.preprocessing.StandardScaler().fit(resha(train_x))  #  標準化後的數據 
    train_x=sk.preprocessing.scale(resha(train_x),0) # normalize each feature
    
    Npdata=train_x
    np.save(os.path.join('./StockData/TrainingData/','NormtrainingX_'+stock_name),Npdata)
    print(path[12:21]," trainX  ",Npdata.shape)
    print(Npdata)
    
    Npdata=scaler.transform(resha(x_test))# normalize x_test with scale of train_x
    np.save(os.path.join('./StockData/TrainingData/','NormtestingX_'+stock_name),Npdata)
    print(path[12:21]," testX  ",Npdata.shape)
    print(Npdata)
    
    Npdata=np.array(train_y)
    np.save(os.path.join('./StockData/TrainingData/','trainingY_'+stock_name),Npdata)
    print(path[12:21]," trainY  ",Npdata.shape)
    print(Npdata)
    
    Npdata=np.array(y_test)
    np.save(os.path.join('./StockData/TrainingData/','testingY_'+stock_name),Npdata)
    print(path[12:21]," testY  ",Npdata.shape)
    print(Npdata)  

'''def MA(data):
    close=data.loc[ : ,'close'].values.tolist()
    ma5=[]
    ma20=[]
    ma60=[]
    for i in range(0,len(close)):
        if i>=4:
            ma5.append(sum(data.loc[ i-4 : i ,'close'].values.tolist())/5)
            if i >= 20 :
                ma20.append(sum(data.loc[ i-20 : i ,'close'].values.tolist())/20)

            if i>= 60 :
                ma60.append(sum(data.loc[ i-60 : i ,'close'].values.tolist())/60) 
        else:
            ma5.append(0)
            ma20.append(0)
            ma60.append(0)
    data['ma5']=ma5
    data['ma20']=ma20 
    data['ma60']=ma60
    return data'''
        
def add_features(data):  # kd   rsv=(今天收盤-最近9天的最低價)/(最近9天的最高價-最近9天的最低價)
    rsv=[] 
    k=[] 
    d=[]
    close=data.loc[ : ,'close'].values.tolist() 
    #print(close)
    for i in range(0,len(close)):
        if i>=8:
            low=min(data.loc[ i-8 : i ,'low'].values.tolist())
            high=max(data.loc[ i-8 : i ,'high'].values.tolist()) 
            # print((close[i]-low)/(high-low))
            rsv.append(((close[i]-low)/(high-low))*100)
        else:
            rsv.append(0)          
    for i in range(0,len(rsv)):
        if i>=1:
            k.append(((2/3)*k[i-1])+((1/3)*rsv[i]))
            d.append(((2/3)*d[i-1])+((1/3)*k[i]))
        else:
            k.append(rsv[0])
            d.append(k[0])            
    data['k']=k
    data['d']=d
    data['rsv']=rsv
    return data

def generate_train(close_type,feature,data):
    train_x=[]
    train_y=[]
    for tmp in data:
        # print(tmp[1])
        i=tmp[1].dropna()
        if len(i)==5: #Decide the way seperate the stock data
            train_x.append(i.loc[ : ,feature].values.tolist())
            #print(train_x)
            Open=i.loc[ : ,'open'].values.tolist()[0]
            Close=i.loc[ : ,close_type].values.tolist()[len(i.loc[ : ,close_type].values.tolist())-1]
            #train_y.append(float("%.2f" % ((Close-Open)/Open)))
            train_y.append(float("%.2f" %((Close-Open))))
        else:
            continue
    print(train_y)
    #exit()
    savenp(train_x,train_y)

feature=input("Enter the needed features seperated with space: ").split(" ")
ctype=input("Enter the close type: ")
resam=input("Enter the desired  data span , d  for day  w  for week  m  for month: ")
#span=int(input("Enter the span of trainx trainy: "))

for path in glob.glob(r'./StockData/stock0050.csv'):
    csv_data=pd.DataFrame(pd.read_csv(path))
    csv_data['date']=pd.to_datetime(csv_data['date'])
    csv_data=csv_data.drop([0],axis=0)#drop 第一天 因為stockdata 有16年跳到17年的問題
    csv_data = add_features(csv_data)
    csv_data=MA(csv_data)
    print(csv_data)
    csv_data=csv_data.set_index('date').resample(resam)#group
    generate_train(ctype,feature,csv_data)

Enter the needed features seperated with space: close open volume k d rsv ma20 ma60
Enter the close type: close
Enter the desired  data span , d  for day  w  for week  m  for month: w


ValueError: Length of values does not match length of index